In [35]:
import pandas as pd

data = pd.read_excel('노래 리스트.xlsx', sheet_name = 'Sheet1')
data

,가수,노래
0,선우정아,도망가자
1,해쉬스완,요즘것들
2,에픽하이,HERE COME THE REGRETS
3,에픽하이,Wish you were
4,타블로,HOOD
5,maroon5,lost
6,에픽하이,트로트
7,코드쿤스트,꽃
8,이적,하늘을 달리다


In [36]:
import selenium
from selenium import webdriver as wd
import time
import pandas as pd
from bs4 import BeautifulSoup
import datetime
import re

# 드라이버 경로 지정
driver = wd.Chrome(executable_path=r'...')
time.sleep(2)

# 윈도우 크기 조절
driver.maximize_window()

In [37]:
# Bugs 홈페이지로 가기
driver.get('https://music.bugs.co.kr/') 

# 검색창 클릭
driver.find_element_by_xpath('//*[@id="headerSearchInput"]').click()

In [38]:
from tqdm import tqdm
import ssl
import urllib.request
from urllib.request import urlopen

lyrics_list = []

for i in tqdm(range(len(data))):
    # 가수와 노래 값 받기
    singer = data['가수'][i]
    song_name = data['노래'][i]
    
    # 검색어 창에 보내기
    all_key = singer + ' ' + song_name
    driver.find_element_by_xpath('//*[@id="headerSearchInput"]').send_keys(all_key)
    
    driver.find_element_by_xpath('//*[@id="hederSearchFormButton"]').click()
    # 노래 있으면 들어가기
    try:
        driver.find_element_by_xpath('//*[@id="DEFAULT0"]/table/tbody/tr[1]/td[3]/a').click()
        
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
        
        # 가사 가져오기
        lyrics = soup.find('div', attrs={'class':'lyricsContainer'}).text
        
        # 간단한 전처리
        
        # '~님이 가져오신 가사입니다~~~' 없애기'
        idx = lyrics.split(' ').index('님이')
        end_word = lyrics.split(' ')[idx-1]
        end_word_idx = lyrics.find(end_word)

        speical_end = end_word.find('\n')
        lyrics = lyrics[:end_word_idx+speical_end]
        
        # 온점 바꾸기
        lyrics = lyrics.replace('\n', '.')
        
        # 첫 부분 글자만 남기게 (.For나 ' For' -> For)
        s = lyrics.split()
#        res = "".join(re.findall("[a-zA-Z]+", s[0]))
        s[0] = re.sub('[^A-Za-z0-9가-힣]', '', s[0])
        lyrics = ' '.join(s)

        # 마지막에 점 추가
        lyrics = lyrics+'.'
        lyrics = lyrics.replace("\'", "")

    except:
        lyrics = ''
        
    lyrics_list.append(lyrics)
    
    # 앨범 사진
    imge = driver.find_element_by_xpath('//*[@id="container"]/section[1]/div/div[1]/div/ul/li/a/img').get_attribute('src')

    context = ssl._create_unverified_context()


    with urlopen(imge, context=context) as f:
        with open('img/'+str(singer)+'_'+str(song_name)+'.jpg', 'wb') as h:
            img = f.read()
            h.write(img)




100%|█████████████████████████████████████████████| 9/9 [00:41<00:00,  4.64s/it]


In [39]:
driver.quit()

In [43]:
# 영어 스펠링 체크 > 한국어로 번역 > 한국어 맞춤법 교정
from hanspell import spell_checker
from gingerit.gingerit import GingerIt
import math

import googletrans
translator = googletrans.Translator()
finish = []

for i in tqdm(range(len(lyrics_list))):
    result = []
    ori_target = lyrics_list[i]
    lyric_length = len(ori_target)
    need_list = math.ceil(lyric_length / 399) # 1.4개의 리스트가 필요하면 2개가 필요하다고 본다.

    for i in range(need_list):
        # 영어 스펠링 체크
        target = ori_target[i*400:(i+1)*400]
        target = target.replace("\'", "")
        try:
            corrected_text1 = GingerIt().parse(target)
            to_kor = corrected_text1['result']
        except:
            to_kor = target

        # 한국어로 번역하기
        translated = translator.translate(to_kor, dest='ko').text
        #print(translated)

        # 한국어 맞춤법 교정
        result.append(spell_checker.check(translated).checked)

    finish.append(",".join(result))

100%|█████████████████████████████████████████████| 9/9 [02:26<00:00, 16.30s/it]


In [46]:
ori = pd.DataFrame(lyrics_list, columns =['원가사'])
ori

,원가사
0,도망가자어디든 가야 할 것만 같아.넌 금방이라도 울 것 같아.괜찮아.우리 가자.걱정...
1,젊은 놈들 전부 벌어가게 사장들 앞에.아저씨 stop it 해 랩이나 내 발밑이라도...
2,Honest to god.I dont remember why.I woke up on...
3,For the last few years.Its been pourin rain.So...
4,Where I’m from.Han is the name we gave to .str...
5,"Had no connection, no faith or direction, no.S..."
6,하아 힘든 세상.어디 하나 기댈 데도 없는.이 세상.너뿐이다 트로트.아무리 각 잡아...
7,아름다워 Like a flower.Life is like a flower.상을 탔어...
8,두근거렸지 .누군가 나의 뒤를 쫓고 있었고 .검은 절벽 끝 .더 이상 발 디딜 곳 ...


In [47]:
prepro = pd.DataFrame(finish, columns = ['1차 전처리 후'])
prepro

,1차 전처리 후
0,도망가자 어디든 가야 할 것만 같아. 넌 금방이라도 울 것 같아. 괜찮아. 우리 가...
1,젊은 놈들 전부 벌어가게 사장들 앞에. 아저씨 stop it 해 랩이나 내 발밑이라...
2,정직한 하나님 께. 나는 그 이유를 기억하지 못한다. 나는 바닥에 일어났습니다. 4...
3,지난 몇 년 동안. 비가 쏟아져 나왔습니다. 그래서 나는 집에 머물러 왔습니다. 내...
4,Where I’m from. Han is the name we gave to. St...
5,"연결이 없고 믿음이나 방향 없음, 아니오. 내 영혼을 구하기 위해 누군가를 검색하고..."
6,하아 힘든 세상. 어디 하나 기댈 데도 없는. 이 세상. 너뿐이다 트로트. 아무리 ...
7,아름다워 Like a flower. Life is like a flower. 상을 ...
8,두근거렸지. 누군가 나의 뒤를 쫓고 있었고. 검은 절벽 끝. 더 이상 발 디딜 곳 ...


In [48]:
df = pd.concat([data, ori, prepro], axis=1)
df

,가수,노래,원가사,1차 전처리 후
0,선우정아,도망가자,도망가자어디든 가야 할 것만 같아.넌 금방이라도 울 것 같아.괜찮아.우리 가자.걱정...,도망가자 어디든 가야 할 것만 같아. 넌 금방이라도 울 것 같아. 괜찮아. 우리 가...
1,해쉬스완,요즘것들,젊은 놈들 전부 벌어가게 사장들 앞에.아저씨 stop it 해 랩이나 내 발밑이라도...,젊은 놈들 전부 벌어가게 사장들 앞에. 아저씨 stop it 해 랩이나 내 발밑이라...
2,에픽하이,HERE COME THE REGRETS,Honest to god.I dont remember why.I woke up on...,정직한 하나님 께. 나는 그 이유를 기억하지 못한다. 나는 바닥에 일어났습니다. 4...
3,에픽하이,Wish you were,For the last few years.Its been pourin rain.So...,지난 몇 년 동안. 비가 쏟아져 나왔습니다. 그래서 나는 집에 머물러 왔습니다. 내...
4,타블로,HOOD,Where I’m from.Han is the name we gave to .str...,Where I’m from. Han is the name we gave to. St...
5,maroon5,lost,"Had no connection, no faith or direction, no.S...","연결이 없고 믿음이나 방향 없음, 아니오. 내 영혼을 구하기 위해 누군가를 검색하고..."
6,에픽하이,트로트,하아 힘든 세상.어디 하나 기댈 데도 없는.이 세상.너뿐이다 트로트.아무리 각 잡아...,하아 힘든 세상. 어디 하나 기댈 데도 없는. 이 세상. 너뿐이다 트로트. 아무리 ...
7,코드쿤스트,꽃,아름다워 Like a flower.Life is like a flower.상을 탔어...,아름다워 Like a flower. Life is like a flower. 상을 ...
8,이적,하늘을 달리다,두근거렸지 .누군가 나의 뒤를 쫓고 있었고 .검은 절벽 끝 .더 이상 발 디딜 곳 ...,두근거렸지. 누군가 나의 뒤를 쫓고 있었고. 검은 절벽 끝. 더 이상 발 디딜 곳 ...


In [49]:
df.to_csv('크롤링한 가사.csv', encoding='utf-8-sig')

- 단점: 영어반, 한국어 반인 경우 영어가 번역되지 않고 거의 input 그대로 나온다.
- 번역할 때 src를 지정 안해서 지금 자동으로 언어 캐치해서 바꿔주기 때문. 
- 하지만, src='eng'로 지정하면 위 단점이 해결되지만 (나머지 절반 영어가 번역됨), 한국어->한국어가 이상하게 나온다. 예를 들어 '나는 하늘을 날고 싶다' -> '나는 날난'?? 정확히 기억 안 나는데 대충 이렇게 원문장과 매우 동떨어진 결과가 나온다.
- 결론: 2차 가공에서는 한글이랑 온점정도만 남기고 다 지우기..? (힘들다.)

# 아래는 과정

## 영어 가사 -> 한국어 가사로 변환
- 바로 번역기를 돌려보았지만, 성능이 매우 나쁨을 볼 수 있었다. 이는 가사의 특성상 띄어쓰기와 줄바꿈, 문법 등이 비교적 자유로워서 좋지 못한 번역 결과가 나온 것으로 판단 된다. 때문에, 영어 스펠링 체크를 통해 번역 성능을 비교해본다.

- 번역기는 googletrans를 이용하였다. (큰 이유는 없고 간편해서)

In [174]:
import googletrans
translator = googletrans.Translator()

## 에픽하이의 Wish you were. (왜냐면 가사가 짧고.. 노래가 좋아서!!!)

# 1) 원본, 번역기
print(lyrics_list[3])
print()
print(translator.translate(lyrics_list[3], dest='ko').text)

.For the last few years.It's been pourin' rain.So I've been a stay home dad.My daughter saw the pain.And whispered Stay home Dad.And people's take on that is.That I'm washed up.Well dear.All my peers are drowning in their past.So I'd rather be.Right

. 지난 몇 년 동안. 나는 쏟아져 나왔다. 나는 vie를 낳았다. 나는 집에 머물러 있었다. 내 딸은 고통을 보았다. 그리고 속삭였다. 아빠들을 속삭였다. 그리고 사람들이 씻어 내고있다.Dear.All 내 동료들은 그들의 과거에 익사하고 있습니다. 나는 오히려 낳을 것입니다.


In [175]:
tt = " For the last few years It's been pourin' rain. So I've been a stay home dad. My daughter saw the pain. And whispered Stay home. Dad. And people's take on that is. That I'm washed up Well dear. All my peers are drowning in their past. So I'd rather be Right here. "

# 2) 수작업으로 온점 찍기, 번역기
print(tt)
print()
print(translator.translate(tt, dest='ko').text)

 For the last few years It's been pourin' rain. So I've been a stay home dad. My daughter saw the pain. And whispered Stay home. Dad. And people's take on that is. That I'm washed up Well dear. All my peers are drowning in their past. So I'd rather be Right here. 

지난 몇 년 동안 그것은 비가 내리고 있습니다.그래서 나는 집에 머물러 왔습니다.내 딸이 고통을 보았습니다.그리고 속삭였다.아빠.사람들이 그걸 가져 가야합니다.내가 잘 씻겨 졌는지.모든 동료들은 과거에 익사하고 있습니다.그래서 나는 오히려 여기있을 것입니다.


In [188]:
# 3) TextBlob이라는 영어 스펠링 체크 적용

from textblob import TextBlob

b = TextBlob(lyrics_list[3])
checked_b = str(b.correct())

print(lyrics_list[3])
print()
print(checked_b)
print()
print(translator.translate(checked_b, dest='ko').text)

.For the last few years.It's been pourin' rain.So I've been a stay home dad.My daughter saw the pain.And whispered Stay home Dad.And people's take on that is.That I'm washed up.Well dear.All my peers are drowning in their past.So I'd rather be.Right

.For the last few years.It's been pouring' rain.To I've been a stay home dad.By daughter saw the pain.And whispered Stay home Had.And people's take on that is.That I'm washed up.Well dear.All my peers are drowning in their past.To I'd rather be.Right

지난 몇 년 동안 지난 몇 년 동안 쏟아져 나왔다. 나는 쏟아져 나왔다. 나는 집에 머물러 있었다.Dear.All 내 동료들은 그들의 과거에 익사하고 있습니다.


In [178]:
# 4) GingerIt이라는 영어 스펠링 체크 적용

from gingerit.gingerit import GingerIt

corrected_text = GingerIt().parse(lyrics_list[3])
new_result = corrected_text['result']

print(lyrics_list[3])
print()
print(new_result)
print()
print(translator.translate(new_result, dest='ko').text)

.For the last few years.It's been pourin' rain.So I've been a stay home dad.My daughter saw the pain.And whispered Stay home Dad.And people's take on that is.That I'm washed up.Well dear.All my peers are drowning in their past.So I'd rather be.Right

. For the last few years. It's been pouring' rain. So I've been a stay home dad. My daughter saw the pain. And whispered Stay home Dad. And people's take on that is. That I'm washed up. Well, dear. All my peers are drowning in their past. So I'd rather be. Right

...에지난 몇 년 동안.비가 내리고 있었다.그래서 나는 집에 머물러 왔습니다.내 딸이 고통을 보았습니다.그리고 속삭였다.사람들이 그걸 가져 가야합니다.내가 씻겨 줬어.글쎄, 친애하는.모든 동료들은 과거에 익사하고 있습니다.그래서 나는 오히려있을 것입니다.오른쪽


In [ ]:
## 스펠링 체크하는 것이 더 말이되는 번역이 나온다.
## 하지만 아직도 미흡한 듯하여 줄개행에 해당하는 문자에 변화주어 확인

In [3]:
singer = '에픽하이'
song_name = 'Wish you were'

all_key = singer + ' ' + song_name
driver.find_element_by_xpath('//*[@id="headerSearchInput"]').send_keys(all_key)
driver.find_element_by_xpath('//*[@id="hederSearchFormButton"]').click()
driver.find_element_by_xpath('//*[@id="DEFAULT0"]/table/tbody/tr[1]/td[3]/a').click()
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')
lyrics = soup.find('div', attrs={'class':'lyricsContainer'}).text

idx = lyrics.split(' ').index('님이')
end_word = lyrics.split(' ')[idx-1]
end_word_idx = lyrics.find(end_word)

speical_end = end_word.find('\n')
lyrics = lyrics[:end_word_idx+speical_end]

In [192]:
# 가사
lyrics

"\nFor the last few years\nIt's been pourin' rain\nSo I've been a stay home dad\nMy daughter saw the pain\nAnd whispered Stay home Dad\nAnd people's take on that is\nThat I'm washed up\nWell dear\nAll my peers are drowning in their past\nSo I'd rather be\nRight here"

In [194]:
# 줄개행 대신 온점
first = lyrics.replace('\n', '.')
# 줄개행 대신 띄어쓰기
second = lyrics.replace('\n', ' ')

In [199]:
# TextBlob으로 비교
from textblob import TextBlob

b = TextBlob(first)
c = TextBlob(second)
print(str(b.correct()))
print()
print(translator.translate(str(b.correct()), dest='ko').text)
print()

print(str(c.correct()))
print()
print(translator.translate(str(c.correct()), dest='ko').text)

.For the last few years.It's been pouring' rain.To I've been a stay home dad.By daughter saw the pain.And whispered Stay home Had.And people's take on that is.That I'm washed up.Well dear.All my peers are drowning in their past.To I'd rather be.Right here

지난 몇 년 동안 지난 몇 년 동안 쏟아져 나왔다. 나는 쏟아져 나왔다. 나는 집에 머물러 있었다. 딸이 고통을 보았다. 그리고 그 밖에서 누워 있었다. 그리고 사람들이 씻어 내고있는 사람들이 씻어 내고있다.Dear.All 내 동료들이 그들의 과거에 익사하고 있습니다. 나는 여기에있을 것입니다.

 For the last few years It's been pouring' rain To I've been a stay home dad By daughter saw the pain And whispered Stay home Had And people's take on that is That I'm washed up Well dear All my peers are drowning in their past To I'd rather be Right here

지난 몇 년 동안 그것은 쏟아지는 것입니다. 나는 딸이 고통을 겪고있는 집에 머물고있는 집에 머물러 있고, 사람들이 가지고있는 사람들이 씻어 내고있는 모든 동료들이 모든 동료들이 익사하고 있다는 것입니다.나는 오히려 여기에 있고 싶습니다


In [201]:
# GingerIt으로 비교
from gingerit.gingerit import GingerIt

corrected_text1 = GingerIt().parse(first)
corrected_text2 = GingerIt().parse(second)

print(corrected_text1['result'])
print()
print(translator.translate(corrected_text1['result'], dest='ko').text)
print()

print(corrected_text2['result'])
print()
print(translator.translate(corrected_text2['result'], dest='ko').text)

. For the last few years. It's been pouring' rain. So I've been a stay home dad. My daughter saw the pain. And whispered Stay home Dad. And people's take on that is. That I'm washed up. Well, dear. All my peers are drowning in their past. So I'd rather be. Right here

...에지난 몇 년 동안.비가 내리고 있었다.그래서 나는 집에 머물러 왔습니다.내 딸이 고통을 보았습니다.그리고 속삭였다.사람들이 그걸 가져 가야합니다.내가 씻겨 줬어.글쎄, 친애하는.모든 동료들은 과거에 익사하고 있습니다.그래서 나는 오히려있을 것입니다.바로 여기에

 For the last few years It's been pouring' rain So I've been a stay home dad My daughter saw the pain And whispered Stay home Dad And people's take on that is That I'm washed up Well dear All my peers are drowning in their past So I'd rather be Right here

지난 몇 년 동안 그것은 쏟아져 나오는 비가 쏟아져 나와 딸이 고통을 보았고 집에 머물러 있고 사람들이 그걸로 씻어내는 것이 잘 씻겨 졌다는 것입니다.나는 오히려 여기에있을 것입니다


In [247]:
## 공통적으로 줄개행 대신 온점 찍는 것이 더 말이 되는 것 같다. (주관적)
## 하지만 앞부분이랑 뒤를 더 신경써야될것 같아서 전처리 추가

# 첫부분에 글자만
s = first.split()
res = "".join(re.findall("[a-zA-Z]+", s[0]))
s[0] = res
first = ' '.join(s)

# 마지막에 점 추가
first = first+'.'

first

"For the last few years.It's been pourin' rain.So I've been a stay home dad.My daughter saw the pain.And whispered Stay home Dad.And people's take on that is.That I'm washed up.Well dear.All my peers are drowning in their past.So I'd rather be.Right here.."

In [248]:
c = TextBlob(first)
print(str(c.correct()))
print()
print(translator.translate(str(c.correct()), dest='ko').text)
print()

corrected_text1 = GingerIt().parse(first)
print(corrected_text1['result'])
print()
print(translator.translate(corrected_text1['result'], dest='ko').text)

For the last few years.It's been pouring' rain.To I've been a stay home dad.By daughter saw the pain.And whispered Stay home Had.And people's take on that is.That I'm washed up.Well dear.All my peers are drowning in their past.To I'd rather be.Right here..

지난 몇 년 동안. 나는 쏟아져 나오고있다. 나는 집에 머물러 있었다. 딸이 고통을 보았다. 그리고 얘들 아가씨.. 모든 동료들은 과거에 익사하고 있습니다. 나는 차라리있을 것입니다.

For the last few years. It's been pouring' rain. So I've been a stay home dad. My daughter saw the pain. And whispered Stay home Dad. And people's take on that is. That I'm washed up. Well, dear. All my peers are drowning in their past. So I'd rather be. Right here..

지난 몇 년 동안.비가 내리고 있었다.그래서 나는 집에 머물러 왔습니다.내 딸이 고통을 보았습니다.그리고 속삭였다.사람들이 그걸 가져 가야합니다.내가 씻겨 줬어.글쎄, 친애하는.모든 동료들은 과거에 익사하고 있습니다.그래서 나는 오히려있을 것입니다.바로 여기에..


In [308]:
## 엄청 마음에 드는 것은 아니지만, 그래도 raw로 번역하는 것보다 더 좋은 성능이 나온다.
## 번역 후 한국어 스펠링 체크 필요

# 영어 스펠링 체크 > 한국어로 번역 > 한국어 문법 및 맞춤법 체크

In [309]:
# 영어 스펠링 체크
target = lyrics_list[1]
print(target)
print()

try:
    corrected_text1 = GingerIt().parse(target)
    to_kor = corrected_text1['result']
except:
    to_kor = target
    print('주로 한국어 가사임')
    print()

# 한국어로 번역하기
translated = translator.translate(to_kor, dest='ko').text
print(translated)


.젊은 놈들 전부 벌어가게 사장들 앞에.아저씨 stop it 해 랩이나 내 발밑이라도 닿게.한 시즌만에 갔지 dab은 so checkin new shit 배로.봐 내 여유 The deep-set eyes 여긴 없지 trend.안따라가도 난 안빠져 눈 굴리는 놈 대답바로.99년생 민증도 못땠지 난 B rabbit 너흰 papadog.어린새끼 유명해 나빠보여 ? okay 그랬겠지 나라도.근데 넌 절대 못해 나 했던대로 두번 태어나봐도.Yeah 난 유행됐고 다 돌려 끼지.귀 밑에 껴놔 홍원's rings.but 내꺼랑 달라 길이.구매처를 알아 Dickids.yeah make up을하는 팀 나의 부름에 답해.young b stan by .삼다수 When i splash..묵묵하시던 아저씨도.꼭 한번씩 뭐라하시죠.옷입은 꼬라지 저 꼬라지 좀 보라지.아예 아예 아 예 예 예.Ah yeah ah yeah yeah yeah.요즘것들 이래서 안 돼요.요즘것들 이래서 안 돼요.각도는 18도로 굽힌채로.아예 아예 아 예 예 예.Ah yeah ah yeah yeah yeah..끝이 없는 배움의 끝 난 대충해도 돼 재능의 부.음 이런 쉽게 살려는 요즘 애들 또 배우겠군.이봐 그럼 안되요 요즘 애들이 이래요 .이런 말을 하면 꼰대래요.풉 풉 풉 메롱.Im stackin stackin money .요즘 은행가면 엄마 카드인줄 아네.야 요즘 애들은 엄마 카들 쓰면서 짝퉁에 민감하다매.야 야 야 꺼져 압구정 매장이 열면 .난 요즘 애들은 비싸 쳐다도 못보는 .가격표의 먼지를 털어.요즘 유행이잖아 총 모양의 손 우리 야망.뒤쳐진것들 플레이리스트에 갖다 대고 빵야 빵야.너도 요즘 애들 되고 싶다면 우선 비와이 형부터 까.요즘 것들은 열등감이 다라는 .내 지난 1년의 연구성과 야..요즘 나 좀 hashtag 섹시.택 시 처럼 타 비 행기.펌핑 댄싱 역기 땡김 청기 백기.이 버르장 머리 없는 새끼.SNS SOS so hot.더블클릭에 좋아요는 heart?!.WIFI LOG ON REFRESH SC

In [310]:
# 한국어 번역
# https://github.com/ssut/py-hanspell/blob/master/hanspell/spell_checker.py
## 긴 가사는 자꾸 False가 나와서 보았더니 최대 500자였다...ㅎㅎ

# 구글 번역도 글자 수 제한이 있더라.
# https://github.com/UlionTse/translators/issues/40

In [311]:
from hanspell import spell_checker
import math

result = []

check_spelling = translated
lyric_length = len(check_spelling)

need_list = math.ceil(lyric_length / 499) # 1.4개의 리스트가 필요하면 2개가 필요하다고 본다.

for i in range(need_list):
    result.append(spell_checker.check(check_spelling[i*500:(i+1)*500]).checked)
    
finish = ",".join(result)

In [312]:
finish

". 젊은 놈들 전부 벌어가게 사장들 앞에. 아저씨 stop it 해 랩이나 내 발밑이라도 닿게. 한 시즌만에 갔지 dab은 so checkin new shit 배로. 봐 내 여유 The deep-set eyes 여긴 없지 trend. 안 따라가도 난 안 빠져 눈 굴리는 놈 대답 바로. 99년생 민증도 못 땠지 난 B rabbit 너흰 papadog. 어린 새끼 유명해 나빠 보여? okay 그랬겠지 나라도. 근데 넌 절대 못해 나 했던 대로 두 번 태어나 봐도. Yeah 난 유행됐고 다 돌려 끼지. 귀밑에 껴놔 홍원's rings.but 내 거랑 달라 길이. 구매처를 알아 Dickids.yeah make up을 하는 팀 나의 부름에 답해. young b stan by. 삼다수 When i splash.. 묵묵하시던 아저씨도. 꼭 한 번씩 뭐라 하시죠. 옷 입은 꼬락서니 저 꼬락서니 좀 보라지. 아예 아예 아 예 예 예. Ah yeah ah yeah yeah yeah. 요즘 것들 이래서 안 돼요. 요즘 것들 이래서 안 돼요. 각도는 18도로 굽힌 채로,.아예 아예 아 예 예 예. Ah yeah ah yeah yeah yeah.. 끝이 없는 배움의 끝난 대충 해도 돼 재능의 부. 음 이런 쉽게 살려는 요즘 애들 또 배우겠군. 이봐 그럼 안돼요 요즘 애들이 이래요. 이런 말을 하면 꼰대래요. 풋 풋 풋 메롱. Im stackin stackin money. 요즘 은행 가면 엄마 카드인 줄 아네. 야 요즘 애들은 엄마 카들 쓰면서 짝퉁에 민감하다며. 야 야 야 꺼져 압구정 매장이 열면. 난 요즘 애들은 비싸 쳐다도 못 보는. 가격표의 먼지를 털어. 요즘 유행이잖아 총 모양의 손 우리 야망. 뒤처진 것들 플레이리스트에 갖다 대고 빵야 빵야. 너도 요즘 애들 되고 싶다면 우선 비와이 형부터 까. 요즘 것들은 열등감이 다라는. 내 지난 1년의 연구성과 야.. 요즘 나 좀 hashtag 섹시. 택 시처럼 타 비 행기. 펌핑 댄싱 역기 땅김 청기 백기. 이 버르장 머리 없는 새